In [69]:
import pandas as pd
from PIL import Image,ImageDraw,ImageFont
import os
import numpy as np
import math

In [70]:
feature_table = pd.read_csv("193398_feattable.csv")

In [71]:
ii = pd.read_csv("193398_individual_importances.csv")

In [72]:
ii.risk_1.value_counts().head()

ts_id_p1y_trafficstopsbysearchreason_consent_sum     299
dispatch_id_p1w_dispatchtype_domestic_disturb_sum    172
dispatch_id_p1w_dispatchtype_domestic_disturb_avg    145
dispatch_id_p5y_dispatchtype_assault_avg             124
irag_id_all_dayssincelastallegation_min              106
Name: risk_1, dtype: int64

In [73]:
ii.risk_2.value_counts().head()

dispatch_id_p1w_dispatchtype_domestic_disturb_avg    159
dispatch_id_p1w_dispatchtype_domestic_disturb_sum    137
dispatch_id_p5y_dispatchtype_assault_avg             116
irag_id_all_dayssincelastallegation_min               90
dispatch_id_p1y_dispatchtype_traffic_sum              76
Name: risk_2, dtype: int64

In [74]:
ii.risk_3.value_counts().head()

dispatch_id_p1w_dispatchtype_domestic_disturb_avg    74
dispatch_id_p1y_dispatchtype_traffic_sum             73
dispatch_id_p1y_dispatchtype_assist_sum              73
shifts_id_p5y_hourspershift_sum                      70
dispatch_id_p5y_dispatchtype_assault_avg             70
Name: risk_3, dtype: int64

In [75]:
ii.risk_4.value_counts().head()

dispatch_id_p1y_dispatchtype_traffic_sum     85
dispatch_id_p1y_dispatchtype_disorder_sum    76
dispatch_id_p1y_dispatchtype_drug_sum        74
dispatch_id_p1y_dispatchtype_shooting_sum    69
shifts_id_p5y_hourspershift_sum              66
Name: risk_4, dtype: int64

In [76]:
ii.risk_5.value_counts().head()

dispatch_id_p1y_dispatchtype_assist_sum             74
dispatch_id_p1y_dispatchtype_suspicious_sum         73
dispatch_id_p1y_dispatchtype_disorder_sum           70
dispatch_id_p1y_dispatchtype_intoxicated_avg        69
dispatch_id_p1w_dispatchinitiatiationtype_ci_sum    68
Name: risk_5, dtype: int64

In [77]:
ii.set_index("entity_id",inplace=True)

In [78]:
feature_table = feature_table[feature_table.feature_importance>0.004]
feature_table.reset_index(drop=True,inplace=True)
top_feats = list(feature_table.feature)

In [79]:
len(top_feats)

5

In [80]:
top_feats

['ocag_id_all_officerage_max',
 'dispatch_id_p1y_dispatchinitiatiationtype_ci_sum',
 'dispatch_id_p1m_dispatchinitiatiationtype_ci_sum',
 'dispatch_id_p1y_dispatchtype_assist_sum',
 'dispatch_id_p1y_dispatchtype_assault_sum']

In [81]:
singlevars = [
    ii.risk_1.loc[1364],
    ii.risk_2.loc[1364],
    ii.risk_3.loc[1364],
    ii.risk_4.loc[1364],
    ii.risk_5.loc[1364]
    ]

In [82]:
singlevars

['ts_id_p1y_trafficstopsbysearchreason_consent_sum',
 'ts_id_p1y_trafficstopsbyrace_black_sum',
 'dispatch_id_p1m_dispatchtype_traffic_sum',
 'dispatch_id_p1y_dispatchtype_traffic_sum',
 'shifts_id_p5y_hourspershift_sum']

In [83]:
#hotvar = top_feats[1]
#cfeat = top_feats[1]
hotvar = singlevars[4]
cfeat = singlevars[4]

#### test matrix

In [84]:
test_matrix = pd.read_csv("193398_test_2014-04-01.csv")
test_matrix.set_index("officer_id",inplace=True)

In [85]:
test_matrix.sort_values(by='score',ascending=False,inplace=True)
test_matrix['idx'] = range(len(test_matrix))
labels = test_matrix.outcome
scores = test_matrix.score

In [86]:
len(test_matrix)

1466

In [87]:
test_matrix = test_matrix[test_matrix['ocag_id_all_officerage_max']!=0]

In [88]:
len(test_matrix)

1465

In [89]:
test_matrix['hotvar_imp'] = False

In [90]:
ii_idxs = [item for item in ii.index if item in test_matrix.index]

In [91]:
for i in ii_idxs:    
    featlist = [
        ii.risk_1.loc[i],
        ii.risk_2.loc[i],
        ii.risk_3.loc[i],
        ii.risk_4.loc[i],
        ii.risk_5.loc[i]
    ]
    
    if hotvar in featlist:
        test_matrix.hotvar_imp.loc[i] = True

In [92]:
print min(scores),max(scores)

0.0624336247543 0.145132730406


In [93]:
test_matrix.outcome.value_counts()

0    1309
1     156
Name: outcome, dtype: int64

In [94]:
tmp = test_matrix[['ocag_id_all_officerage_max','outcome']]

In [95]:
prior = float(156)/1309

In [96]:
max(tmp.ocag_id_all_officerage_max) - min(tmp.ocag_id_all_officerage_max)

45.0

In [97]:
test_matrix.hotvar_imp.value_counts()

False    1130
True      335
Name: hotvar_imp, dtype: int64

In [98]:
hotidxs = test_matrix.index[test_matrix.hotvar_imp==True]

### Browsing

In [106]:
test_matrix[['outcome','score','idx']].sort_values(by="score",ascending=False).head(5)

,outcome,score,idx
officer_id,,,
5443,1,0.145133,0
4519,1,0.144844,1
3537,0,0.143754,2
4879,1,0.142910,3
1465,0,0.141253,4


In [107]:
ii.loc[3537]

model_id                                                193398
as_of_date                                          2014-04-01
risk_1        ts_id_p1y_trafficstopsbysearchreason_consent_sum
risk_2                dispatch_id_p1y_dispatchtype_assault_sum
risk_3               dispatch_id_p1y_dispatchtype_disorder_sum
risk_4               dispatch_id_p1m_dispatchtype_disorder_sum
risk_5        dispatch_id_p1y_dispatchinitiatiationtype_ci_sum
Name: 3537, dtype: object